In [43]:
import tensorflow as tf
from sklearn.datasets import load_diabetes

X_diab, y_diab = load_diabetes(return_X_y=True)

In [44]:
class Layer:
    def __init__(self, units, input_layer=False):
        self.input_layer = input_layer
        self.units = units
        self.activation = 'relu'
        self.grad_w = None
        self.grad_b = None

    @tf.function
    def activationFunction(self, z):
        if self.activation == "linear":
            return z

        if self.activation == "relu":
            return tf.maximum(z, tf.zeros(z.shape))

        if self.activation == "sigmoid":
            return 1 / (1 + tf.exp(-z))

    def _setGrad(self, w, b):
        self.grad_w = w
        self.grad_b = b

    def _updateGrad(self):
        self.w.assign_sub(0.0001 * self.grad_w)
        self.bias.assign_sub(0.0001 * self.grad_b)

    def _weightInit(self, input_size):
        self.w = tf.Variable(tf.random.normal((input_size, self.units), mean = 0, stddev = 1 / input_size, name = 'w'))
        self.bias = tf.Variable(tf.zeros((1, self.units)))

    @tf.function
    def __call__(self, X):
        return self.activationFunction(X @ self.w + self.bias)

In [45]:
class DenseNetwork:
    def __init__(self, layers):
        self.layers = layers

        for i in range(1, len(self.layers)):
            self.layers[i]._weightInit(self.layers[i - 1].units)


    def fit(self, X, y):
        X = tf.constant(X, dtype="float32")
        y = tf.constant(y, dtype="float32")
        
        for _ in range(1000):
            with tf.GradientTape(persistent=True) as tape:
                loss = self.network(X, y)

            for layer in self.layers[1:]:
                layer._setGrad(tape.gradient(loss, layer.w), tape.gradient(loss, layer.bias))

            for layer in self.layers[1:]:
                layer._updateGrad()
                
        del tape

    def predict(self, X):
        X = tf.constant(X, dtype='float32')
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    @tf.function
    def network(self, X, y):
        y_pred = self.forward(X)
        loss = tf.losses.mean_squared_error(y_pred, y)
        tf.print(loss)
        return loss

    @tf.function
    def forward(self, X):
        for layer in self.layers[1:]:
            X = layer(X)
        return X

In [46]:
nn = DenseNetwork([Layer(10), Layer(30),  Layer(1)])

nn.fit(X_diab, y_diab)

nn.predict(X_diab)

[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[28728 28729.4336 28727.9531 ... 28727.791 28728.418 28729.6172]
[23932.6035 23945.2383 23932.9805 ... 23931.5488 23937.873 23943.9336]
[214660.984 214096.234 214657.266 ... 214733.188 214402.062 214102.406]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]
[29074.4824 29074.4824 29074.4824 ... 29074.4824 29074.4824 29074.4824]


<tf.Tensor: shape=(442, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
 